In [11]:
import requests
import json

USERNAME = "bbb"
PASSWORD = "."


In [12]:
def getToken(username, password):
    res = requests.post(
        "http://localhost:8000/get-token/",
        data={"username": "bbb", "password": "."},
        verify=False,
    )
    return res.json()["token"]


def makeRoom(token, title, subtitle=None):
    res = requests.post(
        "http://localhost:8000/rooms/",
        headers={"Authorization": "Token " + TOKEN},
        data={"subtitle": subtitle, "title": title},
    )
    return res


def makeEvent(token, room, author, event_type, args):
    res = requests.post(
        "http://localhost:8000/events/",
        headers={"Authorization": "Token " + TOKEN, "content-type": "application/json"},
        data=json.dumps({"room": room, "author": author, "event_type":event_type, "args":args}),
    )
    return res

def makeTune(token):
    with open("test.mp3", 'rb') as fobj:
        res = requests.post(
            "http://localhost:8000/tunes/",
            headers={"Authorization": "Token " + TOKEN},
            files={'file': fobj},
        )
        return res
    
def getEvents(token, room, event_type, skip, limit):
    url =  "http://localhost:8000/events/"
    
    res = requests.get(url,
                       params={"room":room, "event_type":event_type, "skip":skip, "limit":limit }, 
                       headers={"Authorization": "Token " + TOKEN})
    return res

def getTuneMeta(token, tune_id):
    url = "http://localhost:8000/tunes/{}/meta/".format(tune_id)
    res = requests.get(url, headers={"Authorization": "Token " + TOKEN})
    return res

def getTuneData(token, tune_id):
    url = "http://localhost:8000/tunes/{}/data/".format(tune_id)
    res = requests.get(url, headers={"Authorization": "Token " + TOKEN})
    return res

def getUser(token, skip, limit):
    url = "http://localhost:8000/users/?skip={}&limit={}".format(skip, limit)
    res = requests.get(url, headers={"Authorization": "Token " + TOKEN})
    return res


TOKEN = getToken(USERNAME, PASSWORD)



SEX

In [6]:
res = makeRoom(TOKEN, "cwnewr", "boop")
res.json()

{'id': 1, 'title': 'cwnewr', 'subtitle': 'boop', 'creator': 2, 'members': [2]}

In [88]:
res = makeEvent(TOKEN, 1, 2, "M", {"action": "sexss"})

In [3]:
res = makeTune(TOKEN)

In [92]:
res = getTuneMeta(TOKEN, 2)

In [15]:
res = getUser(TOKEN, 0, 5)

In [107]:
res = getTuneData(TOKEN,2)

In [17]:
res.json()

[{'id': 1, 'username': 'aaa'}]

In [35]:
audio = mutagen.File("test.mp3", easy=True)
audio.tags

{'album': ['TunePocket Unlimited Royalty Free Music Library'], 'composer': ['www.tunepocket.com'], 'title': ['Drums Urban Percussion Intro 2 (Music For Video)'], 'artist': ['TunePocket.com Music Library'], 'albumartist': ['www.tunepocket.com'], 'genre': ['Electronic']}

In [39]:
audio.mime[0]

'audio/mp3'

In [40]:
audio.info.length

23.412125

In [44]:
audio["title"]

['Drums Urban Percussion Intro 2 (Music For Video)']